In [1]:
import numpy as np
import pandas as pd
from Load import json_to_df
import pickle
import re

In [2]:
from checklist.editor import Editor
from checklist.pred_wrapper import PredictorWrapper
from checklist.perturb import Perturb
from checklist.test_suite import TestSuite
from checklist.test_types import INV

C:\Users\Capou\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\Capou\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20

In [3]:
with open('baseline_model.pickle', 'rb') as f:
    model = pickle.load(f)
    
def predict_proba(inputs):
    p = np.array([model.predict_proba([x]) for x in inputs]).reshape(-1, 2)
    
    return p
wrapped_pp = PredictorWrapper.wrap_softmax(predict_proba)

C:\Users\Capou\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Capou\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Capou\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.23.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Capou\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator Pipeline from version 

# Invariance, change positive adjectives

In [4]:
df = json_to_df("data\\music_reviews_train.json")

df = df.replace(np.nan, '', regex=True)

#df['reviewText'] = df.reviewText + ' ' + df.summary

data = list(df[df["sentiment"] =="positive"]["reviewText"])

In [5]:
def change_pos_adjectives(x,meta=False, *args, **kwargs):
    
    pos_adjectives = ["good", "nice", "amazing", "perfect", "exciting", "great"]
    ret = []
    ret_meta = []
    for p in pos_adjectives:
        if re.search(r'\b%s\b' % p, x):
            ret.extend([re.sub(r'\b%s\b' % p, p2, x) for p2 in pos_adjectives if p != p2])
            ret_meta.extend([(p, p2) for p2 in pos_adjectives if p != p2])
            
    if meta:
        return ret, ret_meta
    else:
        return ret

In [6]:
ret= Perturb.perturb(data, change_pos_adjectives)

In [7]:
test1 = INV(**ret, labels=1)

In [8]:
test1.run(wrapped_pp)

Predicting 121190 examples


In [9]:
test1.summary(n=2)

Test cases:      17740
Fails (rate):    161 (0.9%)

Example fails:
0.6 Sounded great!
0.5 Sounded good!
0.5 Sounded exciting!

----
0.5 download arrived in good shape, although out of all the downloads many of them do not play all the way to end.
0.6 download arrived in perfect shape, although out of all the downloads many of them do not play all the way to end.
0.6 download arrived in amazing shape, although out of all the downloads many of them do not play all the way to end.

----


# Invariance, change negative adjectives

In [10]:
df = json_to_df("data\\music_reviews_train.json")

df = df.replace(np.nan, '', regex=True)

#df['reviewText'] = df.reviewText + ' ' + df.summary

data = list(df[df["sentiment"] =="negative"]["reviewText"])

In [11]:
def change_neg_adjectives(x,meta=False, *args, **kwargs):
    
    neg_adjectives = ["bad","awful","abhorrent","annoying","poor","inferior","dreadful","nasty","terrible"]
    ret = []
    ret_meta = []
    for p in neg_adjectives:
        if re.search(r'\b%s\b' % p, x):
            ret.extend([re.sub(r'\b%s\b' % p, p2, x) for p2 in neg_adjectives if p != p2])
            ret_meta.extend([(p, p2) for p2 in neg_adjectives if p != p2])
            
    if meta:
        return ret, ret_meta
    else:
        return ret

In [12]:
ret= Perturb.perturb(data, change_neg_adjectives)

In [13]:
test2 = INV(**ret, labels=0)

In [14]:
test2.run(wrapped_pp)

Predicting 55250 examples


In [15]:
test2.summary()

Test cases:      5514
Fails (rate):    161 (2.9%)

Example fails:
0.5 Horrible cd.  This reminds me of the music in movies right before during or after something bad happens. Not soothing at all.  It stressed me out.
0.4 Horrible cd.  This reminds me of the music in movies right before during or after something awful happens. Not soothing at all.  It stressed me out.
0.4 Horrible cd.  This reminds me of the music in movies right before during or after something terrible happens. Not soothing at all.  It stressed me out.

----
0.3 this awful.  not the stevie nicks i remember!
0.6 this bad.  not the stevie nicks i remember!
0.6 this abhorrent.  not the stevie nicks i remember!

----
0.4 Love the song, but this downloaded song goes from high to low and so on, through out the whole song! So annoying.
0.5 Love the song, but this downloaded song goes from high to low and so on, through out the whole song! So bad.

----


# Invariance, changing media format 1

In [40]:
df = json_to_df("data\\music_reviews_train.json")

df = df.replace(np.nan, '', regex=True)

#df['reviewText'] = df.reviewText + ' ' + df.summary

data = list(df[df["sentiment"] =="positive"]["reviewText"])

In [41]:
def change_media(x,meta=False, *args, **kwargs):
    
    media = ["album","cd","song","track","mixtape","cut","tune","melody","ditty","shanty","anthem","hymn","carol","ballad"]
    ret = []
    ret_meta = []
    for p in media:
        if re.search(r'\b%s\b' % p, x):
            ret.extend([re.sub(r'\b%s\b' % p, p2, x) for p2 in media if p != p2])
            ret_meta.extend([(p, p2) for p2 in media if p != p2])
            
    if meta:
        return ret, ret_meta
    else:
        return ret

In [42]:
ret= Perturb.perturb(data, change_media)

In [43]:
test3 = INV(**ret)

In [44]:
test3.run(wrapped_pp)

Predicting 490267 examples


In [45]:
test_cases3 = sorted(test3.to_raw_examples(), key=len, reverse=True)

In [79]:
poscase = [test_cases3[i] for i in range(0, 1000, 25)]

# Invariance, changing media format 0

In [48]:
df = json_to_df("data\\music_reviews_train.json")

df = df.replace(np.nan, '', regex=True)

#df['reviewText'] = df.reviewText + ' ' + df.summary

data = list(df[df["sentiment"] =="negative"]["reviewText"])

In [49]:
ret= Perturb.perturb(data, change_media)

In [50]:
test4 = INV(**ret)

In [51]:
test4.run(wrapped_pp)

Predicting 320263 examples


In [52]:
test_cases4 = sorted(test4.to_raw_examples(), key=len, reverse=True)

In [74]:
negcases =  [test_cases4[i] for i in range(0, 1000, 25)]

In [80]:
output_neg = pd.DataFrame(columns=["reviewText"])

In [81]:
output_neg["reviewText"] = negcases
output_neg["sentiment"] = "negative"
output_neg

,reviewText,sentiment
0,(2.5stars) When I open my rather large storage...,negative
1,(2.5stars) When I open my rather large storage...,negative
2,(2.5stars) When I open my rather large storage...,negative
3,(2.5stars) With the virtual logjam of solid (b...,negative
4,(2.5stars) With the virtual logjam of solid (b...,negative
5,"Unlike most of the 1-star reviewers, my issue ...",negative
6,"Unlike most of the 1-star reviewers, my issue ...",negative
7,"Unlike most of the 1-star reviewers, my issue ...",negative
8,My first encounter with Black Veil Brides was ...,negative
9,My first encounter with Black Veil Brides was ...,negative


In [82]:
output_pos = pd.DataFrame(columns=["reviewText"])

In [83]:
output_pos["reviewText"] = poscase
output_pos["sentiment"] = "positive"
output_pos

,reviewText,sentiment
0,Why? I don't know. The GRAND CANYON mixtape ...,positive
1,Why? I don't know. The GRAND CANYON cut is o...,positive
2,"... Anyway, if you'd like to know more about t...",positive
3,If everyday you could jump on a plane and jet ...,positive
4,If everyday you could jump on a plane and jet ...,positive
5,"Fortunately, due to the rather majestic and ex...",positive
6,"Fortunately, due to the rather majestic and ex...",positive
7,I often find myself speaking about just how Im...,positive
8,I often find myself speaking about just how Im...,positive
9,I often find myself speaking about just how Im...,positive


In [114]:
with open("CheckListing\\difficult_cases.txt", "r") as file:
    output_johan = pd.read_csv(file, sep=";")

In [115]:
output_johan

,reviewText,sentiment
0,This is not a bad taste.,positive
1,This is not a terrible experience.,positive
2,This is not a terrible list.,positive
3,This is not an awful mistake.,positive
4,This is not a great performance.,negative
5,This is not a terrible song.,positive
6,This is not a terrible choice.,positive
7,This is not a terrible tragedy.,positive
8,This is not a great one.,negative
9,This is not a bad surprise.,positive


In [116]:
df_output = pd.concat((output_neg,output_pos,output_johan))

In [113]:
df_output["sentiment"][80:90]

0     postive
1    positive
2    positive
3    positive
4    negative
5    positive
6    positive
7    positive
8    negative
9    positive
Name: sentiment, dtype: object

In [103]:
import json

In [117]:
json.loads(df_output.to_json(orient="records"))

[{'reviewText': "(2.5stars) When I open my rather large storage of Sizzla Kalonji albums collected over the years (and I've got just about every one of them on vinyl as well) there are tons and tons of highlights of the good, bad and downright odd varieties. When you release albums in such a seriously high level of prolificacy through such a wide variety (and quality) of labels, strange things are certain to happen and musically, me, being even amongst the staunchest of Kalonji fans would have to admit that sometimes, he just isn't on point. On the lower end of that scale are the slew of virtually non-descript albums he's had such as Hosanna (which featured two instrumental tracks right in the middle of the album as if the producer were auditioning his music for other artists or something); Brighter Day whose longevity with me lasted about ten days (I seriously don't know if I've put it back on my players ONE TIME since; and Stay Focus which, due to its rather odd set between dancehall

In [118]:
with open("hardcases.json","w+") as file:
    for i in json.loads(df_output.to_json(orient="records")):
        file.write(json.dumps(i)+"\n")

In [119]:
inputPath = 'hardcases.json'

for lineIdx, line in enumerate(open(inputPath)):
    try:
        data = json.loads(line)
    except ValueError as e:
        print('error, instance ' + str(lineIdx+1) + ' is not in valid json format')
        continue
    if 'reviewText' not in data:
        print("error, instance " + str(lineIdx+1) + ' does not contain key "reviewText"')
        continue
    if 'sentiment' not in data:
        print("error, instance " + str(lineIdx+1) + ' does not contain key "sentiment"')
        continue
    if data['sentiment'] not in ['positive', 'negative']:
        print("error, instance " + str(lineIdx+1) + ': sentiment is not positive/negative')
        continue
        
if lineIdx+1 < 100:
    print('Too little instances(' + str(lineIdx) + '), please generate more')
if lineIdx+1 > 1000:
    print('Too many instances(' + str(lineIdx) + '), please generate more')